In [1]:
import os
import re
import random
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from collections import Counter

In [ ]:
fp=open("/content/drive/MyDrive/Colab Notebooks/positive.review", 'r')
fn=open("/content/drive/MyDrive/Colab Notebooks/negative.review", 'r')
filesdata=[]
pcount=0
ncount=0
sentiments=[]
def getdata(line):
  data=[]
  words=line.split(" ")[:-1]
  for word in words:
    w=word.split(":")[0]
    count=int(word.split(":")[1])
    while count>0:
      data.append(w)
      count-=1
  return " ".join(data)
for line in fp.readlines():
  pcount+=1
  filesdata.append(getdata(line))
  sentiments.append(0)
for line in fn.readlines():
  ncount+=1
  filesdata.append(getdata(line))
  sentiments.append(1)
random.seed(0)
temp=list(zip(filesdata, sentiments))
random.shuffle(temp)
filesdata, sentiments=zip(*temp)
filesdata, sentiments=list(filesdata),list(sentiments)
dict_for_words={}
fp=open("/content/drive/MyDrive/Colab Notebooks/positive.review", 'r')
fn=open("/content/drive/MyDrive/Colab Notebooks/negative.review", 'r')
for line in fp.readlines():
  tokens=line.split(" ")
  for word in tokens[:-1]:
    name, counts = word.split(":")
    if name not in dict_for_words:
        dict_for_words[name] = 0
    dict_for_words[name] += int(counts)
for line in fn.readlines():
  tokens=line.split(" ")
  for word in tokens[:-1]:
    name, counts = word.split(":")
    if name not in dict_for_words:
        dict_for_words[name] = 0
    dict_for_words[name] += int(counts)
print(len(dict_for_words))
vectorizer = CountVectorizer()
filesdata[:1]
bow_matrix=vectorizer.fit_transform(filesdata)
bow_matrix[:1]
X_train, X_test, y_train, y_test = train_test_split(bow_matrix, sentiments, test_size=0.2)
c_train=Counter(y_train)
c_test=Counter(y_test)
print("class pos train",c_train[0])
print("class neg train",c_train[1])
print("class pos test",c_test[0])
print("class neg test",c_test[1])
print(X_train.shape)
def MLP(sizes,activation,num_epochs):
   classifier=MLPClassifier(hidden_layer_sizes=sizes, activation=activation, solver='adam', learning_rate='constant', learning_rate_init=0.001, max_iter=num_epochs)
   classifier.fit(X_train,y_train)
   pred=classifier.predict(X_test)
   print(len(sizes)," layers  ",activation)
   print(classification_report(y_test,pred))
   print()
MLP([100],"relu",100)
MLP([100,100],"relu",100)
MLP([100,100,100],"relu",100)
MLP([100],"tanh",100)
MLP([100,100],"tanh",100)
MLP([100,100,100],"tanh",100)
MLP([100],"logistic",100)
MLP([100,100],"logistic",100)
MLP([100,100,100],"logistic",100)

In [ ]:
!wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip glove.6B.zip

In [3]:

words = dict()
def add_to_dict(d, filename):
  with open(filename, 'r') as f:
    for line in f.readlines():
      line = line.split(' ')

      try:
        d[line[0]] = np.array(line[1:], dtype=float)
      except:
        continue

add_to_dict(words, '/content/glove.6B.100d.txt')

In [ ]:
def message_to_word_vectors(message, word_dict=words):
  processed_list_of_tokens = message.split()

  vectors = []

  for token in processed_list_of_tokens:
    if token not in word_dict:
      continue
    
    token_vector = word_dict[token]
    vectors.append(token_vector)
  
  return np.array(vectors, dtype=float)
all_word_vector_sequences = []

for d in filesdata :
  message_as_vector_seq = message_to_word_vectors(d)
    
  if message_as_vector_seq.shape[0] == 0:
    message_as_vector_seq = np.zeros(shape=(1, 100))
  all_word_vector_sequences.append(message_as_vector_seq)
sequence_lengths = []
for i in range(len(all_word_vector_sequences)):
  sequence_lengths.append(len(all_word_vector_sequences[i]))
# import matplotlib.pyplot as plt
# plt.hist(sequence_lengths)
# print(pd.Series(sequence_lengths).describe())

from copy import deepcopy

def pad_X(X, desired_sequence_length=2073):
  X_copy = deepcopy(X)

  for i, x in enumerate(X):
    x_seq_len = x.shape[0]
    sequence_length_difference = desired_sequence_length - x_seq_len
    
    pad = np.zeros(shape=(sequence_length_difference,100))

    X_copy[i] = np.concatenate([x, pad])
  
  return np.array(X_copy).astype(float)

X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(all_word_vector_sequences, sentiments, test_size=0.2)
X_train_new = pad_X(X_train_new)
x,y,z=X_train_new.shape
X_train_new=X_train_new.reshape((x,y*z))
X_test_new = pad_X(X_test_new)
x,y,z=X_test_new.shape
X_test_new=X_test_new.reshape((x,y*z))
classifier=MLPClassifier(hidden_layer_sizes=(100,100), activation="relu", solver='adam', learning_rate='constant', learning_rate_init=0.001, max_iter=100)
classifier.fit(X_train_new,y_train_new)
pred=classifier.predict(X_test_new)
print(classification_report(y_test_new,pred))